In [1]:
import requests
import pandas as pd
from io import StringIO
import re
from collections import defaultdict

In [2]:
# parse the superpathway list, keeping only the first section (i.e. metabolism)
# from brite 08901
# # get superpathway list
# url = 'https://rest.kegg.jp/get/br:br08901'
# resp_str = requests.get(url).text

# # cached
with open('superpathway_cached.txt') as f:
    resp_str = f.read()

p = "\nA.+?\n(.+?)\nA"
matches = re.findall(p, resp_str, re.DOTALL)
match_section = matches[0].split('\n')
pathway_mapping = []
curr_b = ''
p2 = re.compile("^([ABC])\\s+?(\\S.+?)$")
p3 = re.compile("^([0-9]+?)\\s+?(\\S.+?)$")
for row in match_section:
    k, v = re.match(p2, row).groups(0)
    if k == 'B':
        curr_b = v
    if k == 'C':
        n, m = re.match(p3, v).groups(0)
        pathway_mapping.append((str(n), m, curr_b))
pathway_df = pd.DataFrame.from_records(pathway_mapping, columns=['number', 'name', 'superpathway'])
pathway_df.to_csv('../pathways/pathway_manifest.csv', index=None)